Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Data Preparation and Setup
---

This notebook walks you through all the necessary steps to configure your environment and data for this solution accelerator including:

1. Connect to your workspace
2. Deploying a compute cluster for training and forecasting
3. Create, split, and register Datasets used in this accelerator

### Prerequisites
If you have already run the [00_Setup_AML_Workspace](../00_Setup_AML_Workspace.ipynb) notebooks you are all set.

## 1.0 Connect to your Workspace
In the [00_Setup_AML_Workspace](../00_Setup_AML_Workspace.ipynb) notebook you created a [Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace.workspace?view=azure-ml-py). 

In [ ]:
from azureml.core.workspace import Workspace
ws = Workspace.from_config() 

# Take a look at Workspace
ws.get_details()

## 2.0 Create compute

In this step we create an compute cluster that will be used for the training and forecasting pipelines. This is a one-time set up so you won't need to re-run this in future notebooks.

We create a STANDARD_D13_V2 compute cluster. D-series VMs are used for tasks that require higher compute power and temporary disk performance. This [page](https://docs.microsoft.com/en-us/azure/cloud-services/cloud-services-sizes-specs) will gives you more information on VM sizes to help you decide which will best fit your use case.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpucluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D13_V2',
                                                           min_nodes=0,
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## 3.0 Create Datasets

This solution accelerator uses simulated orange juice weekly sales data from [Azure Open Datasets](https://azure.microsoft.com/en-us/services/open-datasets/) to walk you through the process of training many models on Azure Machine Learning. You can learn more about the dataset [here](https://azure.microsoft.com/en-us/services/open-datasets/catalog/sample-oj-sales-simulated/). 

The full dataset includes simulared sales for 3,991 stores with 3 orange juice brands each thus allowing 11,973 models to be trained to showcase the power of the many models pattern. Each series contains data from '1990-06-14' to '1992-10-01'.

We'll start by downloading the first 10 files but you can easily edit the code below to train all 11,973 models.

In [ ]:
#%pip install --upgrade azureml-opendatasets

In [ ]:
import os
from azureml.core.dataset import Dataset
from azureml.opendatasets import OjSalesSimulated

# Pull all of the data
oj_sales_files = OjSalesSimulated.get_file_dataset()

# Pull only the first 10 files
oj_sales_files_small = OjSalesSimulated.get_file_dataset().take(10)

# Create a folder to download
target_path = 'oj_sales_data' 
if not os.path.exists(target_path):
    os.mkdir(target_path)

# Download the data
oj_sales_files_small.download(target_path, overwrite=True)

We will now split each dataset in two parts: one will be used for training in the [training notebook](02_Training_Pipeline.ipynb), and the other will be used for simulating batch inferencing in the [forecasting notebook](03_Forecasting_Pipeline.ipynb). The training files will contain the data records before '1992-5-28' and the last part of each series will be stored in the inferencing files.

Finally, we will upload both sets of data files to the Workspace's default [Datastore](https://docs.microsoft.compython/api/azureml-core/azureml.core.datastore(class)).

In [ ]:
from scripts.helper import split_data_upload_to_datastore

# Connect to default datastore
datastore = ws.get_default_datastore()

# Set upload paths for train and inference splits
ds_train_path = target_path + '_train'
ds_inference_path = target_path + '_inference'

# Provide name of timestamp column in the data and date from which to split into the inference dataset
timestamp_column = 'WeekStarting'
split_date = '1992-05-28'

# Split each file and upload both sets to the datastore 
split_data_upload_to_datastore(target_path, timestamp_column, split_date, datastore, ds_train_path, ds_inference_path)

Next, we create and register [datasets](https://docs.microsoft.com/en-us/azure/machine-learning/concept-data#datasets) in Azure Machine Learning for the train and inference sets. 

Using a [FileDataset](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.file_dataset.filedataset?view=azure-ml-py) is currently the best way to take advantage of the many models pattern so we create FileDatasets in the next cell. We also [register](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-register-datasets#register-datasets) the Datasets in your Workspace; this associates the train/inference sets with simple names that can be easily referred to later on when we train models and produce forecasts. 

In [ ]:
# Create file datasets
ds_train = Dataset.File.from_files(path=datastore.path(ds_train_path), validate=False)
ds_inference = Dataset.File.from_files(path=datastore.path(ds_inference_path), validate=False)

# Register the file datasets
dataset_name = 'oj_data_small'
train_dataset_name = dataset_name + '_train'
inference_dataset_name = dataset_name + '_inference'
ds_train.register(ws, train_dataset_name, create_new_version=True)
ds_inference.register(ws, inference_dataset_name, create_new_version=True)

Now that you've set up your Workspace and created Datasets, move on to 02_Training_Pipeline.ipynb to train and score the models.